## Study on Fix Gamma.

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

In [2]:
resnet50 = models.resnet50(pretrained=True)

In [3]:
bns=[bn for bn in list(resnet50.state_dict().keys()) if 'bn' in bn and 'weight' in bn]

In [4]:
bns_weight=[weight for weight in bns if "weight" in weight]

将所有的bn weight取log2(abs())然后画出来：

In [6]:
import pandas as pd

In [9]:
for i in bns_weight:
    #plt.figure()
    a = np.clip(np.round(np.log2(np.abs(resnet50.state_dict()[i]))),-10,0)
    print(i,pd.Series(a).value_counts().sort_index())
    #sns.distplot(np.round(np.log2(np.abs(resnet50.state_dict()[i]))),kde=False)
    #plt.title(i)
    #plt.show()

bn1.weight -10.0     1
-3.0      4
-2.0     50
-1.0      9
dtype: int64
layer1.0.bn1.weight -10.0     5
-3.0     27
-2.0     31
-1.0      1
dtype: int64
layer1.0.bn2.weight -10.0     3
-9.0      1
-3.0     37
-2.0     23
dtype: int64
layer1.0.bn3.weight -10.0    32
-9.0      2
-8.0      8
-7.0      5
-6.0      9
-5.0      8
-4.0     34
-3.0     69
-2.0     88
-1.0      1
dtype: int64
layer1.1.bn1.weight -10.0     4
-3.0     28
-2.0     32
dtype: int64
layer1.1.bn2.weight -3.0    29
-2.0    35
dtype: int64
layer1.1.bn3.weight -10.0    21
-9.0     20
-8.0     17
-7.0     11
-6.0      9
-5.0     13
-4.0     53
-3.0     90
-2.0     22
dtype: int64
layer1.2.bn1.weight -3.0    26
-2.0    38
dtype: int64
layer1.2.bn2.weight -3.0     4
-2.0    60
dtype: int64
layer1.2.bn3.weight -10.0    26
-9.0     22
-8.0     30
-7.0     23
-6.0     13
-5.0     11
-4.0     35
-3.0     54
-2.0     41
-1.0      1
dtype: int64
layer2.0.bn1.weight -3.0     25
-2.0    103
dtype: int64
layer2.0.bn2.weight -3.0    

In [ ]:
class FixGamma(nn.Module):
    def __init__(self, bntype):
        super(FixGamma, self).__init__()
        self.bntype = bntype

    def forward(self, input):
        data=input.data
        if self.bntype:
            input.data[:int(len(data)/2)]=data[:int(len(data)/2)]*2**-2
            input.data[int(len(data)/2):]=data[:int(len(data)/2)]*2**-3
        else:
            input.data[:int(len(data)/8)]=input.data[:int(len(data)/8)]*2**-2
            input.data[int(len(data)/8):3*int(len(data)/8)]=input.data[int(len(data)/8):3*int(len(data)/8)]*2**-3
            input.data[3*int(len(data)/8):6*int(len(data)/8)]=input.data[3*int(len(data)/8):6*int(len(data)/8)]*2**-4
            input.data[6*int(len(data)/8):]=input.data[6*int(len(data)/8):]*2**-5
        idx = torch.randperm(input.nelement())
        output = input.view(-1)[idx].view(input.size())
        return output

class testDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,num):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.num=num
        self.alldata=torch.randn(num,3,255,255)

    def __len__(self):
        return self.num

    def __getitem__(self, idx):
        temp=self.alldata[idx]
        return temp

class testbn(nn.Module):
    def __init__(self,bntype):
        super(testbn, self).__init__()
        self.bntype = bntype
        self.bn1=nn.BatchNorm2d(3)
        self.fix=FixGamma(self.bntype)
    def forward(self,input):
        out=self.bn1(input)
        out=self.fix(out)
        return out

td=testDataset(100)
dataloader = DataLoader(td, batch_size=4,
                        shuffle=True, num_workers=0)

testmodel=testbn(False)

for data in dataloader:
    testmodel(data)